In [124]:
import os
import pandas as pd

from skp.toolbox.functions import create_double_cv

In [125]:
nih_df = pd.read_csv("/mnt/stor/datasets/chexmask/OriginalResolution/ChestX-Ray8.csv")
chex_df = pd.read_csv("/mnt/stor/datasets/chexmask/OriginalResolution/CheXpert-small.csv")

In [126]:
nih_orig = pd.read_csv("/mnt/stor/datasets/nih-cxr/Data_Entry_2017_v2020.csv")
chex_orig = pd.read_csv("/mnt/stor/datasets/chexpert/CheXpert-v1.0-small/train.csv")
chex_orig["Path"] = chex_orig.Path.apply(lambda x: x.replace("CheXpert-v1.0-small/", ""))

In [127]:
nih_df = nih_df.merge(nih_orig, on="Image Index")
chex_df = chex_df.merge(chex_orig, on="Path", how="right")

In [ ]:
chex_df["view"] = ""
chex_df.loc[chex_df["AP/PA"] == "AP", "view"] = "AP"
chex_df.loc[chex_df["AP/PA"] == "PA", "view"] = "PA"
chex_df.loc[chex_df["Frontal/Lateral"] == "Lateral", "view"] = "L"
chex_df = chex_df.loc[chex_df["view"] != ""]
chex_df.view.value_counts(dropna=False)

In [ ]:
chex_df["Sex"] = chex_df["Sex"].apply(lambda x: x[0])
chex_df = chex_df.loc[chex_df["Sex"] != "U"]
chex_df["Sex"].value_counts(dropna=False)

In [130]:
nih_df["patient"] = nih_df["Patient ID"].astype("str")
chex_df["patient"] = chex_df["Path"].apply(lambda x: x.split("/")[1])

In [131]:
cols = ["patient", "filename", "dice_rca_mean", "right_lung", "left_lung", "heart", "view", "age", "female"]
nih_df = nih_df[["patient", "Image Index", "Dice RCA (Mean)", "Right Lung", "Left Lung", "Heart", "View Position", "Patient Age", "Patient Gender"]]
chex_df = chex_df[["patient", "Path", "Dice RCA (Mean)", "Right Lung", "Left Lung", "Heart", "view", "Age", "Sex"]]

nih_df.columns = cols
chex_df.columns = cols

In [132]:
nih_df["filename"] = nih_df.filename.apply(lambda x: os.path.join("nih-cxr/images/", x))
chex_df["filename"] = chex_df.filename.apply(lambda x: os.path.join("chexpert/CheXpert-v1.0-small/", x))

In [ ]:
df = pd.concat([nih_df, chex_df])
df.head()

In [ ]:
view_dict = {"AP": 0, "PA": 1, "L": 2}
female_dict = {"M": 0, "F": 1}
df["view"] = df["view"].map(view_dict)
df["female"] = df["female"].map(female_dict)
print(df["view"].value_counts(dropna=False))
print(df["female"].value_counts(dropna=False))

In [ ]:
df["mask_present"] = ~df.heart.isna()
df["mask_present"].value_counts()

In [ ]:
df.head()

In [ ]:
df = create_double_cv(df, "patient", 5, 5)
df.head()

In [138]:
df.to_csv("/mnt/stor/datasets/chexmask/train_nih_chexpert_combined_rle_masks_with_age_view_sex_kfold.csv", index=False)